In [1]:
import pandas as pd
import os
import glob
import pprint

In [2]:
# Define the directory containing the TSV files
directory = "claude_3.5_sonnet/"

In [3]:
# Find all TSV files in the directory
tsv_files = glob.glob(os.path.join(directory, "*.tsv"))


In [4]:
pprint.pprint(tsv_files)

['claude_3.5_sonnet/1-s2.0-S0016706121007540-main.tsv',
 'claude_3.5_sonnet/bell-et-al-2020-metatranscriptomic-sequencing-of-a-cyanobacterial-soil-surface-consortium-with-and-without-a-diverse.tsv',
 'claude_3.5_sonnet/alteio-et-al-2020-complementary-metagenomic-approaches-improve-reconstruction-of-microbial-diversity-in-a-forest-soil.tsv',
 'claude_3.5_sonnet/acp-23-15783-2023.tsv',
 'claude_3.5_sonnet/41564_2019_Article_449.tsv',
 'claude_3.5_sonnet/2022.12.12.520098v2.full.tsv',
 'claude_3.5_sonnet/BISS_article_20637.tsv',
 'claude_3.5_sonnet/blair-et-al-2018-exploration-of-the-biosynthetic-potential-of-the-populus-microbiome.tsv',
 'claude_3.5_sonnet/file.tsv']


In [5]:
# Initialize a list to store the DataFrames
all_dfs = list()

# Iterate through the TSV files
for file_path in tsv_files:
    try:
        print(f"Processing {file_path}")
        # Load the TSV file without treating "NA" as NaN
        df = pd.read_csv(file_path, sep='\t', keep_default_na=False)

        # Clean column names for easier access
        df.columns = ["Number", "Item", "Vote", "Comments"]

        # Add a filename column
        df['Filename'] = os.path.basename(file_path)

        # Append the DataFrame to the list
        all_dfs.append(df)

    except Exception as e:
        print(f"Error processing {file_path}: {e}")

Processing claude_3.5_sonnet/1-s2.0-S0016706121007540-main.tsv
Processing claude_3.5_sonnet/bell-et-al-2020-metatranscriptomic-sequencing-of-a-cyanobacterial-soil-surface-consortium-with-and-without-a-diverse.tsv
Processing claude_3.5_sonnet/alteio-et-al-2020-complementary-metagenomic-approaches-improve-reconstruction-of-microbial-diversity-in-a-forest-soil.tsv
Processing claude_3.5_sonnet/acp-23-15783-2023.tsv
Processing claude_3.5_sonnet/41564_2019_Article_449.tsv
Processing claude_3.5_sonnet/2022.12.12.520098v2.full.tsv
Processing claude_3.5_sonnet/BISS_article_20637.tsv
Processing claude_3.5_sonnet/blair-et-al-2018-exploration-of-the-biosynthetic-potential-of-the-populus-microbiome.tsv
Processing claude_3.5_sonnet/file.tsv


In [6]:
# Concatenate all DataFrames into a single DataFrame
combined_df = pd.concat(all_dfs, ignore_index=True)


In [7]:
combined_df

,Number,Item,Vote,Comments,Filename
0,1.0,Structured or unstructured abstract,Yes,Unstructured abstract present on p.1 including...,1-s2.0-S0016706121007540-main.tsv
1,1.1,Study design,Yes,Study design described in abstract - laborator...,1-s2.0-S0016706121007540-main.tsv
2,1.2,Environmental & sample information,Yes,Abstract describes permafrost aggregates from ...,1-s2.0-S0016706121007540-main.tsv
3,1.3,Host information (if applicable),NA,No host organisms involved in this study,1-s2.0-S0016706121007540-main.tsv
4,1.4,Experiments & omics methods,Yes,Abstract describes experimental freeze-thaw cy...,1-s2.0-S0016706121007540-main.tsv
...,...,...,...,...,...
598,14.1,Funding,Yes,Funding sources acknowledged,file.tsv
599,15.0,Conflicts of interest,Yes,Competing interests statement provided,file.tsv
600,16.0,Supplementary data & files,Yes,Supplementary information referenced,file.tsv
601,17.0,Sample & data availability,Yes,Data availability statement provided with DOI,file.tsv


In [15]:
combined_df.to_csv("streams_summary.tsv", sep='\t', index=False)

In [9]:
# Create a file-wise summary
file_summary = combined_df.pivot_table(index='Filename', columns='Vote', aggfunc='count', fill_value=0)['Number']
file_summary = file_summary.reset_index()
file_summary.columns.name = None
file_summary = file_summary.rename(columns={'Filename': 'Filename', 'No': 'No Votes', 'Yes': 'Yes Votes', 'NA': 'NA Votes'})


In [10]:
file_summary

,Filename,NA Votes,No Votes,Yes Votes
0,1-s2.0-S0016706121007540-main.tsv,12,8,47
1,2022.12.12.520098v2.full.tsv,8,7,52
2,41564_2019_Article_449.tsv,6,6,55
3,BISS_article_20637.tsv,44,6,17
4,acp-23-15783-2023.tsv,11,9,47
5,alteio-et-al-2020-complementary-metagenomic-ap...,8,6,53
6,bell-et-al-2020-metatranscriptomic-sequencing-...,8,11,48
7,blair-et-al-2018-exploration-of-the-biosynthet...,7,10,50
8,file.tsv,11,9,47


In [11]:
file_summary.to_csv('streams_summary_by_pdf.tsv', index=False, sep='\t')

In [12]:
# Create a question-wise summary (using Number instead of Item)
question_summary = combined_df.pivot_table(index='Number', columns='Vote', aggfunc='count', fill_value=0)['Item']
question_summary = question_summary.reset_index()
question_summary.columns.name = None
question_summary = question_summary.rename(columns={'Number': 'Question Number', 'No': 'No Votes', 'Yes': 'Yes Votes', 'NA': 'NA Votes'})

# Create a mapping from Number to Item
number_to_item = combined_df.groupby('Number')['Item'].first().to_dict()

# Add the Item column to question_summary using the mapping
question_summary['Item'] = question_summary['Question Number'].map(number_to_item)

# Reorder columns to put 'Item' next to 'Question Number'
question_summary = question_summary[['Question Number', 'Item', 'Yes Votes', 'No Votes', 'NA Votes']]


In [13]:
question_summary

,Question Number,Item,Yes Votes,No Votes,NA Votes
0,1.0,Structured or unstructured abstract,9,0,0
1,1.1,Study design,9,0,0
2,1.2,Environmental & sample information,9,0,0
3,1.3,Host information (if applicable),1,0,8
4,1.4,Experiments & omics methods,9,0,0
...,...,...,...,...,...
62,14.1,Funding,8,1,0
63,15.0,Conflicts of interest,5,4,0
64,16.0,Supplementary data & files,7,2,0
65,17.0,Sample & data availability,8,0,1


In [14]:
question_summary.to_csv('streams_summary_by_question.tsv', index=False, sep='\t')